In [4]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
import librosa
import os

# Définir le dossier contenant les fichiers audio
folder_path = os.path.expanduser("~/Music/PFA II/db_8khz/A")

# Définir la liste des extensions de fichiers audio à traiter
audio_ext = (".wav", ".mp3", ".m4a")

# Définir la liste des caractéristiques à extraire
feature_names = ["mfcc{}".format(i) for i in range(1, 14)]
#Les accolades {} seront remplacées par la valeur de i,qui est insérée dans la chaîne à l'aide de la méthode .format().



# Initialize the DataFrame to store the feature data
data = pd.DataFrame(columns=["name", "status"] + feature_names)

# Boucle chaque fichier audio dans le dossier.
for filename in os.listdir(folder_path):
    if filename.endswith(audio_ext):
        file_path = os.path.join(folder_path, filename)
        name = os.path.splitext(filename)[0]
        status = 1 if name.startswith("P") else 0  # Les fichiers commençant par "P" sont considérés comme appartenant 
        #à des patients atteints de la maladie de Parkinson, et tous les autres fichiers sont considérés comme sains.
        
        # Extract MFCC features
        signal, sr = librosa.load(file_path, sr=8000)
        
        mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13, hop_length=int(len(signal)/2), window='hann', lifter=22, n_mels=22)
        mfccs_mean = np.mean(mfccs.T, axis=0) #pour calculer la moyenne de chaque coefficient sur toutes les trames du signal audio.
        
        # Ajouter les données des caractéristiques au DataFrame
        row = [name, status] + list(mfccs_mean)
        data.loc[len(data)] = row

# Save the DataFrame to a CSV file
data.to_csv("audio_features_A.csv", index=False)
